In [ ]:
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer
import time

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

## Gathering Data

In [ ]:
df_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
df_archive.head()

In [ ]:
# Create dataframe from TSV
df_image_predictions = pd.read_csv('image-predictions.tsv', delimiter='\t')

In [ ]:
df_image_predictions.head()

In [ ]:
#gather data from twitter 
# Twittter App Credentials
consumer_key = "A0lPFhv6c1twbEz8BY4r4TfOb"
consumer_secret = "zQXzmROzedXF9UaoV18siNc7ftNhXMrBXK8uAC4F54fdDfWbbh"
access_token = "1179780108988477440-5DySAvAMwAemnn5UmvXsFIJW3It8Av"
access_secret = "k8dQUxpIdLAuWeWzBYtP3xYfumvkjbmNynDCG7CEjVNQW"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [ ]:
tweet_id_list = df_archive['tweet_id']

In [ ]:
start = time.time()
print("started")

df_list = []
for tweet_id in tweet_id_list:
    try:
        result = api.get_status(tweet_id, tweet_mode='extended')
        retweet_count = result['retweet_count']
        favorite_count = result['favorite_count']
        df_list.append({'tweet_id': tweet_id, 'retweet_count' : retweet_count, 'favorite_count': favorite_count})
    except Exception as e:
        print (tweet_id)

end = time.time()
print(end - start)

In [ ]:
# Save only certain tweet elements in dataframe
elements_to_save = ['id', 'favorite_count', 'retweet_count']

data = []

with open('tweet_json.txt', 'r') as readfile:
    
    tweet_json = readfile.readline()
    
    while tweet_json:
        tweet_dict = json.loads(tweet_json)

        data_row = dict((k, tweet_dict[k]) for k in elements_to_save)
        
        data.append(data_row)
        
        tweet_json = readfile.readline()
        
            
    
df_tweet = pd.DataFrame.from_dict(data)

In [ ]:
df_tweet.head()

### Data:
We now have altogether 3 datasets:
- **df_tweet**- Dataframe with retweet and favorite count
- **df_image_predictions**- Dataframe with image prediction data
- **df_archive** - Dataframe with archived tweets data

## Assessing the data

#### df_archive

In [ ]:
df_archive

In [ ]:
df_archive.info()

In [ ]:
df_archive.describe()

In [ ]:
df_archive.name.value_counts()

**Observation** 

Issue:
- There are names such as "a","an","the" which is very unlikely to be a dog's name.
- Datatype correction is needed (timestamp is shown as object)
- Some entries are retweets (we need to avoid those)
- Some missing values

Tidiness:
- Four columns (doggo, floofer, pupper, and puppo) can be made into one.


#### df_tweet dataframe

In [ ]:
df_tweet

In [ ]:
df_tweet.info()

In [ ]:
df_tweet.describe()

**Observation**

Issues:
- There are some missing data as df_archive have total 2356 rows but df_tweet only has 2331 rows. 

#### df_image_predictions

In [ ]:
df_image_predictions

In [ ]:
df_image_predictions.info()

In [ ]:
df_image_predictions.describe()

**Observation**

Issues: 
- There are some missing data as df_archive has in total 2356 rows but df_image_predictions only has 2075 rows. 
- There also seems to be an inconsistency with the casing for p1,p1,p3 (some are titlecase, some lowercase, etc)

### Final Data

We will be cleaning the data and merging it to make a final dataset to perform our analysis.

Tidiness:
- We will get rid of data columns related to retweets as we are ignoring retweets.

**df_twitter_master** : This Dataframe will store the final data

# Data Wrangling

In [ ]:
df_twitter_master = df_archive.copy()

### df_archive

#### removing the dog name "a", "an" and "the"

In [ ]:
def replace_names(row):
    if (row['name'] == 'a') or (row['name'] == 'an') or (row['name'] == 'the'):        
        #finding the name to replace with a or an or the        
        text = row['text']
        word_list = text.split()
        name = 'None'
        # get name from 'named {}'
        if 'named' in word_list:
            name_pos = word_list.index('named') + 1
            name = word_list[name_pos]
        # get name from 'name is {}'
        elif 'name is' in text:
            # Get the next word after 'name is' (skip 'is')
            name_pos = word_list.index('name') + 2
            name = word_list[name_pos]
        #only replace the 'name' column
        row['name'] = name.replace('.','')
        # Return row whether updated or name
    return row
    

df_twitter_master  = df_twitter_master.apply(replace_names, axis=1)

In [ ]:
df_twitter_master['name'].value_counts()

**Observation**:
The dog name in the data is cleaned.

#### Correcting the datatypes

In [ ]:
#Correcting the timestamp

df_twitter_master.timestamp = pd.to_datetime(df_twitter_master.timestamp)

df_twitter_master.retweeted_status_timestamp = pd.to_datetime(df_twitter_master.retweeted_status_timestamp)

In [ ]:
df_twitter_master.info()

**Observation**:
The timestamp datatype is fixed

#### cleaning data that are retweets

In [ ]:
# Find all the retweets 
retweets = df_twitter_master.text.str.match('^RT @')


In [ ]:

#Getting rid of retweets
df_twitter_master = df_twitter_master[~retweets]

In [ ]:
retweets = df_twitter_master.text.str.match('^RT @')

df_twitter_master[retweets]

#### Combining the columns into one

In [ ]:
#adding a new column
df_twitter_master['dog_stage'] = 'None'


In [ ]:
#function to combine all the columns
def get_dog_stage(row):
    dog_stage = []
    
    if row['doggo'] == 'doggo':
        dog_stage.append('doggo')
    if row['floofer'] == 'floofer':
        dog_stage.append('floofer')
    if row['pupper'] == 'pupper':
        dog_stage.append('pupper')
    if row['puppo'] == 'puppo':
        dog_stage.append('puppo')
        
    if len(dog_stage) < 1: 
        row['dog_stage'] = 'None'
    else: 
        row['dog_stage'] = ','.join(dog_stage)
    
    return row


In [ ]:
df_twitter_master = df_twitter_master.apply(get_dog_stage, axis=1)

# Drop the redundant columns
df_twitter_master =df_twitter_master.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1)

In [ ]:
df_twitter_master.head()

**Observation**:
We have combined all the four columns

### df_tweet

#### Fixing the missing data issue by combining the df_tweet with df_twitter_master and only keeping the data that is in both dataset

In [ ]:
df_tweet.rename(columns={'id':'tweet_id'}, inplace=True)

df_twitter_master = pd.merge(df_twitter_master, df_tweet, how='left', on=['tweet_id'])

In [ ]:
df_twitter_master.info()

**Observation**:
We have got rid of the missing data from df_tweet

### df_image_predictions

#### Fixing the missing data issue by combining the df_image_predictions with df_twitter_master and only keeping the data that is in both dataset

In [ ]:
df_image_predictions.rename(columns={'id':'tweet_id'}, inplace=True)

df_twitter_master = pd.merge(df_twitter_master, df_image_predictions, how='left', on=['tweet_id'])

In [ ]:
df_twitter_master.info()

**Observation**:
We have got rid of the missing data from df_image_predictions

#### Convert case to lower to maintain the consistency in p1,p2,p3

In [ ]:
df_twitter_master['p1'] = df_twitter_master['p1'].str.lower()
df_twitter_master['p2'] = df_twitter_master['p2'].str.lower()
df_twitter_master['p3'] = df_twitter_master['p3'].str.lower()



In [ ]:
df_twitter_master[['p1','p2','p3']].head()

**Observation** : We have made the case consistent

### df_twitter_master

In [ ]:
# Drop all columns that relate to retweet  
retweet_cols = ['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp']
df_twitter_master = df_twitter_master.drop(retweet_cols, axis=1)

In [ ]:
df_twitter_master.info()

### Saving the dataset as csv

In [ ]:
df_twitter_master.to_csv('twitter_archive_master.csv')

# Exploring the cleaned data

In [ ]:
#If we tweet the post of our dog, how many favourite count can we expect?

df_twitter_master.hist(column='favorite_count', bins=20, range=(0,80000));
plt.title('Favorite count')
plt.xlabel('Favorites')
plt.ylabel('Count');

**Observation** : 
Seems like most of the tweets gets around 0 to 10000 and only few tweets seems to break that boundary.

In [ ]:
#If we tweet the post of our dog, how many retweets are we most likely to get?

df_twitter_master.hist(column='retweet_count', bins=25, range=(0,25000));
plt.title('Retweet count')
plt.xlabel('Retweets')
plt.ylabel('Count');

**Observation** : 
The retweet seems to follow the similar pattern to favourite_count. Most of the tweets seems to be retweeted about 0 to 5000 times and only few tweets seems to cross this range.

In [ ]:
#Which stage dogs has maximum tweets?

dog_stage = df_twitter_master[df_twitter_master['dog_stage'] != "None"].dog_stage.value_counts()

ax = (dog_stage).plot(kind='bar', title ="Dog stages", figsize=(15, 10), legend=True, fontsize=20)
ax.set_xlabel("Stage", fontsize=12)
ax.set_ylabel("Count", fontsize=12)
plt.show()



**Observation**: 

Most of the tweets are of dogs in stage ,"pupper"

In [ ]:
# Which breed of dog seems to be most popular?


breed = df_twitter_master[df_twitter_master['p1'] != "None"].p1.value_counts().head(20)

ax = (breed).plot(kind='bar', title ="Dog Breed Popularity", figsize=(15, 10), legend=True, fontsize=20)
ax.set_xlabel("breed", fontsize=12)
ax.set_ylabel("Count", fontsize=12)
plt.show()


**Observation**:
It seems golden retriever and labradors are really popular.

## Conclusion:

After wrangling the data and combining all the data we were able to find many meaningful insights from the data.Such as the range of numbers of retweets and favourite we are likely to get. Also, we were able to find the most popular stage of dog and also the most popular breed of dogs.

## Limitation:
There are alot of insights we can draw out from the data than the insights drawn out in this analysis.